In [1]:
import torch
from matplotlib import pyplot as plt
import torch.nn as nn
import torchvision
import numpy as np
import torchvision.transforms.functional as TF
from torchvision import transforms
%matplotlib inline

In [2]:
#load trained model
device = torch.device('cuda:0')
net2 = torch.hub.load('pytorch/vision:v0.10.0', 'resnext101_32x8d', pretrained=True)
#net= nn.Sequential(r50_swav_200ep,nn.Linear(2048 , 2048),nn.Linear(2048 , 2048),nn.Linear(2048 , 2))
net= nn.Sequential(net2,nn.Linear(1000 , 64),nn.Linear(64 , 64),nn.Linear(64 , 2))
net.to(device)
net.load_state_dict(torch.load("models/test_acc_92_v101_v2.pth"))

Using cache found in /home/srvargh2/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

In [3]:
def test_acc(net,testloader,device):
    correct = 0
    total = 0
    tp=0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images.to(device))
            
            # the class with the highest prob is what we choose as prediction
            #print(outputs.shape)
            predicted = torch.argmax(outputs,dim=1,keepdim=False)
            #print(predicted.shape)
            total += labels.size(0)
            
            #print(labels.shape)
            #print(predicted.shape)
            #print(predicted.to(device) == labels.to(device))
            correct += (predicted.to(device) == labels.to(device)).sum()
            p_lables= 1 == labels.to(device)
            tp+=(predicted.to(device) == p_lables.to(device)).sum()
            

    print(f'Accuracy of the network on the test images: {100 * correct // total} %')
    print(f'number of true positives of the network on the test images: {tp}')
    return int(100 * correct // total)

In [4]:
#test dataloader for acc test
transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                          torchvision.transforms.Resize((250,250)),
                                          torchvision.transforms.RandomHorizontalFlip(),
                                          torchvision.transforms.RandomAdjustSharpness(sharpness_factor=0.5,p=0.25),
                                          torchvision.transforms.RandomAdjustSharpness(sharpness_factor=1.25,p=0.25),
                                          torchvision.transforms.RandomAdjustSharpness(sharpness_factor=2,p=0.25),
                                          torchvision.transforms.RandomPerspective(),
                                          torchvision.transforms.RandomResizedCrop(size=(250,250)),
                                          
                                         ])
#test_dataset=torchvision.datasets.ImageFolder("dataset2/test",transform=transform)
#test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=True)
#temp_acc=test_acc(net,test_loader,device)

In [5]:
from os import listdir
from os.path import isfile, join
image_files = [f for f in listdir("images") if isfile(join("images", f))]
print(image_files[-0])
print(len(image_files))

0011c61f00.jpg
12115


In [6]:
#custom dataset to filter images
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
class TestComponentCustomDataset(Dataset):
    def __init__(self, images_path,image_list):
        super(TestComponentCustomDataset, self).__init__()
        #print(ftrain)
        self.image_path = images_path
        self.trans= transforms.Compose([transforms.ToTensor()])
        self.image_list=image_list
        
    
    def __getitem__(self, index):
        #x is the input
        
        random_img_name=self.image_list[index]
        
        x = Image.open(os.path.join(self.image_path,random_img_name)) 
        x = self.trans(x)

        return x, random_img_name

    def __len__(self):
        return len(self.image_list)
        

test_comp_dataset = TestComponentCustomDataset("images",image_files)
test_comp_dataloader = DataLoader(test_comp_dataset,batch_size=1,shuffle=False)


In [7]:
import shutil
net.eval()
with torch.no_grad():
    for batch,(x,img_name) in enumerate(test_comp_dataloader):
        print(img_name[0])
        y=net(x.to(device))
        preds=y[0].argmax()
        if preds==1:
            src=os.path.join("images",img_name[0])
            dst="filtered_images"
            shutil.copy(src, dst)
            print("human")

0011c61f00.jpg
human
00138e4872.jpg
00145d24fd.jpg
001acafa1d.jpg
001f0a283e.jpg
00240f5868.jpg
0027435611.jpg
human
002c3346ae.jpg
human
003557f3f6.jpg
003644d065.jpg
00534a4248.jpg
0054a5dbce.jpg
human
0055accba3.jpg
human
00581b8c7d.jpg
005a263326.jpg
human
005d6f3baf.jpg
00630cd632.jpg
006b9b7a3c.jpg
0071ae8aaa.jpg
0077223432.jpg
007b37dc70.jpg
human
008103dd64.jpg
009f6ee196.jpg
00a20f3fb5.jpg
human
00a2bdf0a6.jpg
human
00a5a7de72.jpg
00a8c25dfd.jpg
00ad428ad1.jpg
00c1300705.jpg
00c4361a96.jpg
human
00c7873e0f.jpg
human
00d334a3b6.jpg
00d6550b87.jpg
00df71c1b3.jpg
00e3511ce6.jpg
00ed5bc09a.jpg
00f5f018e6.jpg
human
010127e204.jpg
human
0101817efb.jpg
0106794a9f.jpg
010bb6311d.jpg
010c8e1967.jpg
010fd8b603.jpg
0118f7905e.jpg
011a3f5dae.jpg
011afc8f6f.jpg
human
011d0019f3.jpg
011d2ae39f.jpg
human
011ee9d241.jpg
human
012a45970d.jpg
human
012ab79d29.jpg
012ced93bc.jpg
01340421f2.jpg
0134599378.jpg
013a32017d.jpg
human
013e012a29.jpg
0151bed91d.jpg
human
0165b45295.jpg
016881accf.jpg
0

0af7f30817.jpg
0afa708b35.jpg
0afbd9f3d9.jpg
0afdd1c3ef.jpg
0b00f4f389.jpg
human
0b023f4fcb.jpg
0b116d1cb3.jpg
human
0b17fda8c3.jpg
0b1837e6ca.jpg
0b261c0c11.jpg
human
0b2b1eeb49.jpg
0b2ccc0ae9.jpg
0b37ab22e7.jpg
0b462b5066.jpg
0b47bf1e16.jpg
human
0b5ae1833b.jpg
0b5da460f3.jpg
0b5fae59c8.jpg
human
0b651c6838.jpg
0b65bcaa05.jpg
0b670a9442.jpg
0b6cc7a0e7.jpg
human
0b77a81870.jpg
0b7d412b8d.jpg
0b806bbf33.jpg
0b81f8a590.jpg
human
0b86aa22c4.jpg
human
0b8ca97bfd.jpg
human
0ba1eaf04d.jpg
0bab0d2bb1.jpg
human
0bb0a2ee59.jpg
0bb464cc24.jpg
human
0bbbbe3312.jpg
0bbca25f37.jpg
0bbd515bac.jpg
human
0bbf72d676.jpg
human
0bc2d341d4.jpg
0bc7b911cb.jpg
0bd1676443.jpg
0bd83d29bd.jpg
human
0bdcccb611.jpg
0be9494161.jpg
human
0bea5c04c0.jpg
human
0bf1e821be.jpg
0bfec6d290.jpg
0c02ccd804.jpg
human
0c04806a44.jpg
0c048f949c.jpg
0c07265756.jpg
0c08c51c09.jpg
human
0c0c697626.jpg
0c299f4b12.jpg
human
0c2ab2801f.jpg
0c2c48fa99.jpg
0c2cb3b80c.jpg
human
0c3789d502.jpg
human
0c3e8e1a5b.jpg
0c3ef4419b.jpg
huma

14382d66e5.jpg
143aefe2f1.jpg
143f493e68.jpg
144aef8c2b.jpg
14635bb05c.jpg
14645e4fc1.jpg
1476b67975.jpg
14770d4b1a.jpg
human
147fdb7c4e.jpg
148ac418fd.jpg
148c7b50ac.jpg
1493f0fef6.jpg
149c9fe182.jpg
14a1332557.jpg
14a38f7d7d.jpg
14a44bb0f3.jpg
human
14a8bd5551.jpg
14b141527c.jpg
human
14b9d7d144.jpg
human
14c432d69d.jpg
human
14ce73ffea.jpg
14d226cff4.jpg
human
14d88dc021.jpg
14d9aca53a.jpg
14dadb28da.jpg
human
14dcaefa9d.jpg
14e2d73ae1.jpg
human
14e6537345.jpg
14e99f9ac4.jpg
14ed2c39c7.jpg
14ee8f190c.jpg
14ef53236b.jpg
14f0130c84.jpg
human
14fb72bc14.jpg
14fdba36fa.jpg
14fe300efa.jpg
human
15040fdb7c.jpg
1509466c2e.jpg
150b03896a.jpg
human
150d62f39c.jpg
150e02872a.jpg
human
1511d0e68f.jpg
1513165e02.jpg
151c376574.jpg
152f05a02b.jpg
human
153b6b5ee3.jpg
human
155c05d3ac.jpg
156438c6be.jpg
human
15678c1927.jpg
human
156c5d7db2.jpg
156d0fd0dd.jpg
1573d72823.jpg
1574992482.jpg
human
1579cf39c1.jpg
157d4cc919.jpg
15838d3f0c.jpg
1585a58f6a.jpg
1586aab432.jpg
human
1586cf16be.jpg
1596bae

1e26c8378c.jpg
1e27bca912.jpg
human
1e3873bf93.jpg
human
1e3e5860f8.jpg
1e3ebb8d99.jpg
human
1e412618f2.jpg
1e425248d0.jpg
human
1e4c6bb0c8.jpg
1e52b4d1be.jpg
1e59ed864a.jpg
1e63b341af.jpg
human
1e64ac908f.jpg
human
1e6672e11e.jpg
human
1e6e539620.jpg
1e7e589b3e.jpg
1e7f23b6dc.jpg
human
1e819351e0.jpg
1e8835a9e7.jpg
human
1e8d8de8e7.jpg
1e8dedd40c.jpg
human
1e95235d18.jpg
human
1ea8f6456e.jpg
1eaa09f6e3.jpg
1eaa3faa14.jpg
1eaadb9f1a.jpg
1eb224eb1b.jpg
1eb363bc6c.jpg
human
1ece8f1ff0.jpg
human
1ecf16ab30.jpg
1ed47d2991.jpg
human
1ed98c2fba.jpg
1edb94e710.jpg
human
1ee401ad28.jpg
1ee58518cb.jpg
1eed84e4c4.jpg
1ef703769d.jpg
1efa49c48e.jpg
1f051d338d.jpg
1f06f172ff.jpg
1f0795dabb.jpg
human
1f102e2dde.jpg
human
1f132071b6.jpg
1f13c9c419.jpg
1f140ec9b5.jpg
1f28b64c74.jpg
human
1f2937b4b7.jpg
1f2ca3195d.jpg
1f3ca39af3.jpg
1f44eb1a02.jpg
human
1f4a822318.jpg
human
1f4e71216b.jpg
1f53ecbc55.jpg
human
1f553aff17.jpg
human
1f56004ee2.jpg
human
1f57289b6a.jpg
human
1f57c3e140.jpg
1f5fd71e14.jpg
h

281c4738e3.jpg
human
282865f5e7.jpg
2828f60c9c.jpg
human
2831cc0170.jpg
human
283c8cb49c.jpg
28431534f7.jpg
human
2844904be8.jpg
28454814c9.jpg
human
284823d2e3.jpg
human
284aceae3b.jpg
2851ca4341.jpg
2855908731.jpg
human
285c7213e8.jpg
285f605918.jpg
human
28639bdb4d.jpg
human
2864322fed.jpg
human
2868827a24.jpg
286a215960.jpg
human
287eb4e157.jpg
2888786d74.jpg
human
2890e734e4.jpg
289d13b276.jpg
289d77f410.jpg
28a2e0a175.jpg
human
28a6b346ae.jpg
human
28b2365afc.jpg
28b6701e0f.jpg
human
28bed62d2f.jpg
28c096d779.jpg
28c6daa3ab.jpg
human
28cbe7822d.jpg
human
28dc480795.jpg
human
28e4b196e4.jpg
28f4a31678.jpg
28f81f2186.jpg
human
28fb2cafbd.jpg
28fcc5a212.jpg
2901b0ff14.jpg
human
290c8446a2.jpg
human
2911e5a347.jpg
2911ea9eca.jpg
human
2915c04c4d.jpg
291be5205c.jpg
human
29200eb585.jpg
2922fb4519.jpg
human
292cfc7aaf.jpg
human
292f8aff22.jpg
human
2931171b00.jpg
293ee343ab.jpg
2940a0ceb0.jpg
human
2944cba9fa.jpg
294ec7f607.jpg
295ca5f8b5.jpg
human
2961fbe771.jpg
2968170cd4.jpg
human
2

...

d37e53c1d1.jpg
d381509686.jpg
d381810449.jpg
d3893787ef.jpg
human
d38e1f1acb.jpg
human
d38e7589cc.jpg
human
d390c71a0e.jpg
d395dde564.jpg
human
d39612b866.jpg
d39628a77d.jpg
d39be30b20.jpg
human
d39db078f8.jpg
human
d3a03ad0fe.jpg
d3a362751e.jpg
human
d3a42d4209.jpg
human
d3a665add0.jpg
human
d3ac79916e.jpg
human
d3b362be03.jpg
human
d3c2abf7b2.jpg
d3c52bbe6d.jpg
d3c8903ecd.jpg
human
d3d49d6e03.jpg
d3da864981.jpg
human
d3e0805f3b.jpg
human
d3ed9512fd.jpg
human
d3faca0abe.jpg
d3fd473463.jpg
d3feeec2ea.jpg
d400a22a77.jpg
d40809ed91.jpg
d40a4df2ac.jpg
d40f40ef15.jpg
human
d414e5637b.jpg
d415779ecc.jpg
d41ba3a95e.jpg
d41c5bcff7.jpg
d41c9203f3.jpg
d42c35f6c0.jpg
d4314c904b.jpg
human
d4325d58a6.jpg
d433b96c18.jpg
d437ba97de.jpg
d438999d9f.jpg
d43d3bc0b0.jpg
human
d4450ec068.jpg
d4459f4244.jpg
d445dccc25.jpg
d449e4a144.jpg
d44aca31ae.jpg
d454f5336d.jpg
d45745b8b2.jpg
d45880e917.jpg
human
d45a1a4e67.jpg
d45ae0f87c.jpg
human
d46204a2f4.jpg
human
d46c2a7e84.jpg
human
d470c3920f.jpg
d47825b3be.jp

human
ddd95bb6bc.jpg
human
dde2663bad.jpg
dde2823754.jpg
human
dde7d7a452.jpg
human
dde9973359.jpg
ddea487bb0.jpg
ddf8bb2437.jpg
ddfc8bde58.jpg
human
ddfcd1115a.jpg
de02fb636d.jpg
human
de0f73cf88.jpg
human
de0f9b1523.jpg
de1961f9ba.jpg
de1a048558.jpg
human
de21817880.jpg
de27d5e8ce.jpg
de2f184f1b.jpg
human
de338f558b.jpg
de368e0233.jpg
de3a56cfbb.jpg
human
de3bc3f9ca.jpg
de40324a66.jpg
human
de44089d7d.jpg
de4aa4eeaf.jpg
de4ac8d28c.jpg
human
de4ce812f6.jpg
human
de50322459.jpg
de51a0b70a.jpg
de5664ad83.jpg
human
de56b2ab04.jpg
de5fb7195e.jpg
human
de65751c79.jpg
de6b22ad6e.jpg
human
de72da9eb8.jpg
de73623271.jpg
human
de84d080e8.jpg
human
de8b27df71.jpg
human
de8c01f26e.jpg
de8e9ca420.jpg
de9137fd6b.jpg
dea2cf315c.jpg
dea5fdad0c.jpg
human
dea7f04a5d.jpg
human
deab994d73.jpg
human
deb1b4938c.jpg
human
deb3b809a2.jpg
deb6816571.jpg
human
debe4cd57a.jpg
human
dec49371f9.jpg
ded7f8c88e.jpg
deda031a28.jpg
human
dedd1de734.jpg
dee0a7b34a.jpg
human
deed6f0c07.jpg
human
def018b1fc.jpg
def11f6

human
e7b85efc87.jpg
human
e7bb84e370.jpg
e7c4f690fb.jpg
e7c920e3ce.jpg
e7cb43ab81.jpg
human
e7cbc30338.jpg
e7d525806e.jpg
e7d8ecca05.jpg
human
e7e7e403f7.jpg
e7f38a9872.jpg
e7f41e18e6.jpg
e7f435cb32.jpg
human
e7fce99b6d.jpg
human
e7fe71f7e6.jpg
e7febece97.jpg
e809a5bb3b.jpg
e80e356e7a.jpg
e81023ad0a.jpg
human
e816b045a3.jpg
e81ef2ffca.jpg
human
e820a91284.jpg
e8262aebbe.jpg
e8296d2277.jpg
e82fde414c.jpg
e82fe39542.jpg
human
e838980a43.jpg
human
e83e845332.jpg
human
e83f2be5d5.jpg
e8416412f5.jpg
human
e8418d5078.jpg
e84269c297.jpg
e84c39cff9.jpg
human
e85bd8989f.jpg
e861199391.jpg
human
e86305ad95.jpg
e86492d607.jpg
e86a193d6c.jpg
human
e86ed01534.jpg
e87527e4c3.jpg
human
e87e5d0aa6.jpg
human
e87f6f41eb.jpg
e8821885cf.jpg
human
e88a7985a5.jpg
human
e89cc3c776.jpg
e89f1ec41c.jpg
human
e8a56dbbc9.jpg
human
e8a5e17a77.jpg
e8b24048da.jpg
e8b31a7356.jpg
human
e8b64a9817.jpg
human
e8b782542e.jpg
e8bc5e0c27.jpg
human
e8c96416ed.jpg
e8cc22913c.jpg
human
e8cfeed68a.jpg
human
e8d12366b1.jpg
e8d8

f0d5c79935.jpg
f0d8d97295.jpg
f0dc5ba674.jpg
f0e067efac.jpg
human
f0e4a707a1.jpg
human
f0efac7d67.jpg
f0f9f85cba.jpg
human
f107525c86.jpg
human
f119ec1bad.jpg
f123978164.jpg
human
f126175c2b.jpg
f12a68f3f9.jpg
f12b3e9e64.jpg
f12dea2da7.jpg
human
f13560bbf8.jpg
f13b0bd949.jpg
f1436225b1.jpg
f14366dd7d.jpg
f1442e4c07.jpg
f144b0a447.jpg
f145ee8304.jpg
human
f14fffd530.jpg
f15785b246.jpg
f15859528f.jpg
f16c829831.jpg
f16cd0bc59.jpg
human
f17216e07e.jpg
human
f1736ea1c1.jpg
human
f1761a0020.jpg
f179b1b849.jpg
human
f17adc33a5.jpg
human
f17dabc4ea.jpg
human
f18773e9d1.jpg
human
f19fbde2b2.jpg
f1b54cf2c3.jpg
f1c4b490bd.jpg
f1caff41c7.jpg
human
f1cff51c91.jpg
f1d2985909.jpg
f1d4c8a423.jpg
f1d6401f49.jpg
human
f1d88348e4.jpg
f1d9c70235.jpg
f1da492c65.jpg
human
f1e06f8b8b.jpg
f1e1a02dc4.jpg
f1e3138ae4.jpg
f1e34814bd.jpg
human
f1ec2b7100.jpg
human
f1ed538562.jpg
human
f1efb02a55.jpg
f1fcacb7d5.jpg
human
f1fcc30313.jpg
human
f1fd487466.jpg
f1fed41b8e.jpg
human
f20515d3eb.jpg
human
f205c0afbe.jpg
f

human
fac1654638.jpg
human
fac3ad3496.jpg
fac86c7bc8.jpg
human
face45e318.jpg
fad08fc755.jpg
fad3dbad2f.jpg
human
fada281a9b.jpg
fae1e78875.jpg
human
fae41a7461.jpg
fae55fd720.jpg
fae6035e29.jpg
human
faf42a0dc0.jpg
faf49dd64d.jpg
human
faf62eb059.jpg
faf8ee6c5b.jpg
fafa0b4a81.jpg
fafc666041.jpg
human
fb03b9ff16.jpg
fb06b7e147.jpg
human
fb08fc0625.jpg
fb0dc971cc.jpg
human
fb0f8c6fe7.jpg
fb1067345c.jpg
fb1337d728.jpg
fb15698ae4.jpg
fb15aac1e1.jpg
fb1f62866a.jpg
fb20733e39.jpg
human
fb218e2d49.jpg
human
fb22396d0e.jpg
human
fb253b2916.jpg
fb2fc86e99.jpg
human
fb32274eef.jpg
fb34c0fc60.jpg
fb3fcfbd8e.jpg
fb5401f243.jpg
human
fb571d4021.jpg
fb581e7a87.jpg
human
fb62f18b37.jpg
human
fb6949cbbd.jpg
fb6a39ad16.jpg
fb7808bb58.jpg
fb790b99af.jpg
fb84315f86.jpg
fb849c890c.jpg
fb91912812.jpg
fb95c03f19.jpg
human
fb96f6abb7.jpg
fb9703e250.jpg
fb9c946306.jpg
fb9f11acd7.jpg
fb9f6631af.jpg
fba791536d.jpg
fbb13d8365.jpg
human
fbba42a498.jpg
fbc1090de2.jpg
fbc6e9db9a.jpg
fbcd9fcf22.jpg
fbd1db0ea9.jpg
f